In [3]:
# import import_ipynb
# import preprocess.save_params_with_name # preprocess重新加载一次 里面有写大文件的问题  不要这样
import sys
import os
import random
import collections
import numpy as np
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error#均方误差

In [4]:
# GBDT
# 因为决策树较容易过拟合，我们设置树的个数为32，叶子节点数设为30，深度就不设置了，学习率设为0.05

In [11]:
def lgb_pred(tr_path, va_path, _sep = '\t', iter_num = 32):
    # load or create your dataset
    print('Load data...')
    df_train = pd.read_csv(tr_path, header=None, sep=_sep)
    df_test = pd.read_csv(va_path, header=None, sep=_sep)
    
    y_train = df_train[0].values
    y_test = df_test[0].values
    X_train = df_train.drop(0, axis=1).values
    X_test = df_test.drop(0, axis=1).values
    
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    # specify your configurations as a dict
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',         #选择二分类。logistic损失  默认regression
        'metric': {'l2', 'auc', 'binary_logloss'},# 验证集的评估指标
        'num_leaves': 30,   #叶子节点数目:默认31 ， 注: 树深度默认6
#         'max_depth': 7,
        'num_trees': 32,  #这是num_iterations,num_boost_round的别名 默认等于100。多分类问题会构建 num_class * numtrees颗树
        'learning_rate': 0.05,
        'feature_fraction': 0.9,  #特征采样比例
        'bagging_fraction': 0.8,  #样本采样比例
        'bagging_freq': 5,
        'verbose': 0   #调试信息
    }
    
    print('Start training...')
    # train
    # lightgbm支持指定类别特征（思考: 为什么类别的特征在决策树种是否可以按照数值形式处理？ https://www.jianshu.com/p/d07f0b0726da）
    # e.g. 年龄类别特征 https://www.deeplearn.me/1625.html
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=iter_num,   #核心参数
                    valid_sets=lgb_eval,
                    feature_name=["I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","I13","C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13","C14","C15","C16","C17","C18","C19","C20","C21","C22","C23","C24","C25","C26"],
                    categorical_feature=["C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13","C14","C15","C16","C17","C18","C19","C20","C21","C22","C23","C24","C25","C26"],
                    early_stopping_rounds=5)
    
    print('Save model...')
    # save model to file
    gbm.save_model('/media/data/cuixuange/Criteo_dataset/model/lgb_model.txt')
    
    print('Start predicting...')
    # predict
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    # eval
    print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

    return gbm,y_pred,X_train,y_train

In [12]:
gbm,y_pred,X_train ,y_train = lgb_pred('/media/data/cuixuange/Criteo_dataset/preprocess_data/train_lgb.txt', '/media/data/cuixuange/Criteo_dataset/preprocess_data/valid_lgb.txt', '\t', 256)

Load data...
Start training...

/home/cuixuange/CS231N_ENV/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/cuixuange/CS231N_ENV/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['C1', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/cuixuange/CS231N_ENV/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')



[1]	valid_0's l2: 0.188062	valid_0's auc: 0.713479	valid_0's binary_logloss: 0.562619
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.186005	valid_0's auc: 0.722179	valid_0's binary_logloss: 0.557344
[3]	valid_0's l2: 0.184091	valid_0's auc: 0.725031	valid_0's binary_logloss: 0.552496
[4]	valid_0's l2: 0.182378	valid_0's auc: 0.727246	valid_0's binary_logloss: 0.548207
[5]	valid_0's l2: 0.180807	valid_0's auc: 0.72867	valid_0's binary_logloss: 0.544297
[6]	valid_0's l2: 0.17934	valid_0's auc: 0.729812	valid_0's binary_logloss: 0.540666
[7]	valid_0's l2: 0.177943	valid_0's auc: 0.732244	valid_0's binary_logloss: 0.537215
[8]	valid_0's l2: 0.176701	valid_0's auc: 0.733889	valid_0's binary_logloss: 0.534151
[9]	valid_0's l2: 0.17552	valid_0's auc: 0.735173	valid_0's binary_logloss: 0.531226
[10]	valid_0's l2: 0.174417	valid_0's auc: 0.736101	valid_0's binary_logloss: 0.528507
[11]	valid_0's l2: 0.173436	valid_0's auc: 0.736616	valid_0's binary_logloss: 0.

In [7]:
"""
查看每个特征的重要程度
"""
gbm.feature_importance()

# gbm.feature_importance("gain")

array([ 14,   0,  30,   9,  12,  70,  31,  14,  17,   0,  41,   0,  31,
         0,  22,   1,  65,   0,   1,  35,   0,   0,  29,  23,  18, 128,
        23, 159,  14,  14,  57,   2,   1,   2,   0,   8,  45,   0,  12])

In [8]:
gbm.feature_importance("gain")

array([  90396.27114868,       0.        ,   20853.79501343,
          4603.42703247,    9545.50097656,  195913.87060547,
        104861.48678589,   15043.76608276,   14878.26693726,
             0.        ,  204698.6499939 ,       0.        ,
         32445.77528381,       0.        ,   11616.3949585 ,
           386.50201416,   40623.47528076,       0.        ,
           295.59899902,   14361.83802795,       0.        ,
             0.        ,   12148.47796631,    9460.6890564 ,
          9334.94799805,  121146.28167725,   24814.36785889,
         90156.56433105,    7586.67001343,   22500.77676392,
         38134.10675049,     654.42199707,     326.26000977,
          1021.3789978 ,       0.        ,    4392.30096436,
         22593.34103394,       0.        ,    6508.33892822])

In [9]:
def ret_feat_impt(gbm):
    """
    特征的重要性排序
    """
    gain = gbm.feature_importance("gain").reshape(-1, 1) / sum(gbm.feature_importance("gain"))
    col = np.array(gbm.feature_name()).reshape(-1, 1)
    return sorted(np.column_stack((col, gain)),key=lambda x: x[1],reverse=True)


ret_feat_impt(gbm)

[array(['I11', '0.18094051872484868'],
       dtype='<U32'), array(['I6', '0.17317533542015728'],
       dtype='<U32'), array(['C13', '0.10708556724199762'],
       dtype='<U32'), array(['I7', '0.0926908497631191'],
       dtype='<U32'), array(['I1', '0.079904524006007'],
       dtype='<U32'), array(['C15', '0.07969263850542067'],
       dtype='<U32'), array(['C4', '0.0359085547947007'],
       dtype='<U32'), array(['C18', '0.033708112177329164'],
       dtype='<U32'), array(['I13', '0.0286799908570874'],
       dtype='<U32'), array(['C14', '0.021934314624693675'],
       dtype='<U32'), array(['C24', '0.019971068917795015'],
       dtype='<U32'), array(['C17', '0.019889248021403072'],
       dtype='<U32'), array(['I3', '0.018433421457463375'],
       dtype='<U32'), array(['I8', '0.013297727360056875'],
       dtype='<U32'), array(['I9', '0.013151436696989016'],
       dtype='<U32'), array(['C7', '0.012694946560213059'],
       dtype='<U32'), array(['C10', '0.010738477781884805'],
     

In [10]:
"""GBDT 模型参数的保存"""
import pickle
dump = gbm.dump_model()
# save_params_with_name((gbm, dump), '/media/data/cuixuange/Criteo_dataset/model/gbm_dump')     #使用pickle序列化保存
# gbm, dump = load_params_with_name('gbm_dump')


#pickle是一种数据压缩方式
#这里选择保存obj=(model,parameter)
pickle.dump((gbm, dump), open('{}.p'.format('/media/data/cuixuange/Criteo_dataset/model/gbm_dump'), 'wb'))
    